## Transform ASTER Level-2 Data into Tiled Northup Product

This notebook provides a protoype function to convert the Level-2 V4 ASTER Products from their archieved rotated map orientation trasnformation to a northup transformation tiled product based upon the HLS (MGRS) Grid.

The goals include:
1. Convert Rotated ASTER Data to Northup tiled product to support potential time series and mosaicing
2. Preserve input paramters (height, wideth, crs, pixel centers) in order to preserve alignment between northup 
3. Write transformed tifs as some output that can be viewed in a GIS to compare with input data

MGRS Grid is Read in from the [HLS Website](https://hls.gsfc.nasa.gov/products-description/tiling-system/)

As of 12/12/25 this method was tested on AST_07 for one acquisition all bands. Still need to try with the other Level-2 Products (Visable: AST_09, AST_09X, AST_07X) and (Thermal:  AST_09T, AST_08, AST_05).

To do:

1. Instead of writing outputs save in memory and process somehow into some usecase 

In [1]:
import geopandas as gpd
import os
import rioxarray as rxr
import earthaccess
from osgeo import gdal
import pandas as pd
import fiona
from rasterio.transform import from_origin
from rasterio.warp import Resampling
import numpy as np
from shapely.geometry import box
import matplotlib.pyplot as plt
import xarray as xr
import requests
import tempfile
import rasterio

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
fiona.supported_drivers['KML'] = 'rw'

In [2]:
#Set working directory for outputs
os.chdir(r'C:\Users\nroberts\aster\data')

#Link to MGRS Grid KML
mgrs_url  = 'https://hls.gsfc.nasa.gov/wp-content/uploads/2016/03/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

In [3]:
#List of https links for tifs to process
url = [' https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B06.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B07.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B08.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B09.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B01.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B02.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_B03N.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_QA_DataPlane.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_QA_DataPlane2.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane.tif',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane2.tif'

]

In [4]:
#Setup URS Login to authenticate to read in source tifs using Earthaccess
earthaccess.login(persist=True)

gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

In [5]:
#Read in MGRS (HLS) Grid from the HLS Website and return as geopandas dataframe
def load_mgrs(mgrs_url):

    response = requests.get(mgrs_url)

    with tempfile.NamedTemporaryFile(suffix='.kml', delete=False) as tmp:
        tmp.write(response.content)
        tmp_path = tmp.name

    mgrs_grid = gpd.read_file(tmp_path, driver = 'KML')
    mgrs_grid = mgrs_grid.set_crs('EPSG:4326')
    return mgrs_grid

In [6]:
#Read in Source (Rotated) Data and return required variables
def read_rotated(input_file, 
                nodata = 0):
    
    rotated = rxr.open_rasterio(input_file, mask_and_scale=False)
    rotated = rotated.rio.write_nodata(nodata)
    rotated_transform = rotated.rio.transform()
    rotated_crs = rotated.rio.crs

    #return rotated xarray, rotated transformation, rotated_crs, rotated_bounds

    return rotated, rotated_transform, rotated_crs

In [7]:
"""Calculate true bounds of the rotated tif, it does not work to just use the bounds of the rotated
data as GIS flips the bounds north because data is assumed not to be rotated by GIS and by
geopandas
"""
def get_rotated_bounds(rotated, rotated_crs):
    rotated_bounds = rasterio.transform.array_bounds(rotated.rio.width, rotated.rio.height, rotated.rio.transform())
    minx, miny, maxx, maxy = rotated_bounds
    geom = box(minx, miny, maxx, maxy) 
    
    gdf_bounds = gpd.GeoDataFrame({'geometry': [geom]}, crs = rotated_crs)
    gdf_bounds = gdf_bounds.to_crs('epsg:4326')

    return gdf_bounds

In [8]:
#Get intesecting MGRS tiles from MGRS Grid and return the tile bounds
def get_mgrs_tile(mgrs_grid, gdf_bounds, rotated_crs):
    mgrs_tile = gpd.sjoin(mgrs_grid, gdf_bounds, predicate='intersects')

    mgrs_tile = mgrs_tile.to_crs(rotated_crs)
    mgrs_tile['minx'] = mgrs_tile.bounds['minx']
    mgrs_tile['miny'] = mgrs_tile.bounds['miny']
    mgrs_tile['maxx'] = mgrs_tile.bounds['maxx']
    mgrs_tile['maxy'] = mgrs_tile.bounds['maxy']
    
    return mgrs_tile


In [9]:
"""
This is where we create the northup tiled products.

1. Iterate over intersecting tiles geodataframe so we get a tile for each part of the source tif
covered.
2. Get bounds for the bounding box of the MGRS Tile.
3. Calculate the X, Y Resolution we can't just use the resolution from the rotated transformation
because it is several decimal points different then the published resolution of 15, 30, and 90 meters
for VNIR, SWIR and TIR.
4. Calculate shape height.
5. Create new transformation for north up
6. Reproject rotated data using north up transform and with and height for mgrs_tiled data
7. Write out ouputs
8. Add coords that might be useful to create a xarray dataset later (Band Name and MGRS Tile ID)
"""
def make_tiled(rotated, rotated_transform, mgrs_tile, rotated_crs, tif):


    for i, row in mgrs_tile.iterrows():
   
        minx = row['minx']
        maxy = row['maxy']
        maxx = row['maxx']
        miny = row['miny']


    
        res_x = np.sqrt(rotated_transform.a**2 + rotated_transform.b**2)
        res_y = np.sqrt(rotated_transform.d**2 + rotated_transform.e**2)

        out_with = int((maxx - minx)/res_x)
        out_height = int((maxy - miny)/res_y)

        north_up = from_origin(minx, maxy, res_x, res_y)

        mgrs_aster = rotated.rio.reproject(rotated.rio.crs, shape = (out_height, out_with), transform = north_up, resampling = Resampling.bilinear)
        tname = row['Name']
        output_file = tif.split('/')[6][:-4]
        if 'QA' in output_file:
            band_name = '_'.join(output_file.split('_')[5:8])
        else:
            band_name = '_'.join(output_file.split('_')[5:7])
        
        mgrs_aster = mgrs_aster.squeeze('band', drop = True)
        mgrs_aster = mgrs_aster.rename(band_name)
        mgrs_aster = mgrs_aster.assign_coords(tile_id = tname)
        mgrs_aster.rio.to_raster(f'{output_file}_{tname}_{'bounds'}.tif')

    return mgrs_aster

In [11]:
#This will create tiled north up tifs for all tifs in the above list

mgrs_grid = load_mgrs(mgrs_url)

for tif in url:
    print('Transforming ', tif)
    rotated, rotated_transform, rotated_crs, = read_rotated(tif)
    rotated_bounds = get_rotated_bounds(rotated, rotated_crs)
    mgrs_tile = get_mgrs_tile(mgrs_grid, rotated_bounds, rotated_crs)

    mgrs_aster = make_tiled(rotated, rotated_transform, mgrs_tile, rotated_crs,tif)

c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\pyogrio\geopandas.py:275: UserWarning: More than one layer found in 'tmplpc9dv33.kml': 'Features' (default), 'Info'. Specify layer parameter to avoid this warning.
  result = read_func(


Transforming   https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


Transforming  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


...

c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


Transforming  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(


Transforming  https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152300_SRF_VNIR_QA_DataPlane2.tif


c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\rioxarray\rioxarray.py:700: UserWarning: Transform that is non-rectilinear or with rotation found. Unable to recalculate.
  warnings.warn(
